# Imports

In [3]:
!pip install mlflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 17.8 MB 941 kB/s 
     |████████████████████████████████| 79 kB 8.2 MB/s 
     |████████████████████████████████| 62 kB 996 kB/s 
     |████████████████████████████████| 209 kB 66.5 MB/s 
     |████████████████████████████████| 181 kB 53.7 MB/s 
     |████████████████████████████████| 596 kB 25.2 MB/s 
     |████████████████████████████████| 146 kB 59.6 MB/s 
     |████████████████████████████████| 54 kB 3.4 MB/s 
     |████████████████████████████████| 63 kB 2.2 MB/s 
     |████████████████████████████████| 78 kB 8.2 MB/s 
  Created wheel for databricks-cli: filename=databricks_cli-0.16.6-py3-none-any.whl size=112631 sha256=f216cf3a0c063fca5b072a3a8eb5ffb81d1ec87fb6850cc663df723149dabd50
  Stored in directory: /root/.cache/pip/wheels/96/c1/f8/d75a22e789ab6a4dff11f18338c3af4360189aa371295cc934
Successfully built databricks-cli
  Attempting uninstall

In [4]:
!pip install memoized_property

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for memoized-property: filename=memoized_property-1.0.3-py2.py3-none-any.whl size=4198 sha256=1c9f4cca378de00e06d17424cacf3c9cf305e6d4bcf6dd0271060afcc069290a
  Stored in directory: /root/.cache/pip/wheels/de/7b/fe/699bb0345131d630549d98185b87309eb5921d460df9f5562d
Successfully built memoized-property


In [5]:
import mlflow
from mlflow.tracking import MlflowClient
from memoized_property import memoized_property

In [6]:
# Data loading libraries 
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [25]:
# Model set up libraries 
from tensorflow import keras
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Activation, ReLU
from tensorflow.keras.layers import BatchNormalization, Conv2DTranspose, Concatenate
from tensorflow.keras.models import Model, Sequential 
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers

from sklearn.model_selection import train_test_split

In [8]:
import glob
from PIL import Image
from tensorflow.keras.preprocessing import image
import numpy as np

# Global Variables

In [9]:
ROOT_PATH = 'drive/MyDrive/SOLAR_ROOF/'
FOLDERS = ['tiled_test_images', 'tiled_train_images']
FOLDER_PATH = [f'{ROOT_PATH}{folder}/*.tif' for folder in FOLDERS]
MLFLOW_URI = "https://mlflow.lewagon.ai/"
EXPERIMENT_NAME = "[UK] [LONDON] [SOLAR_ROOF] TEST RUN"

IMAGE_SIZE = (160,160)
BATCH_SIZE = 8
EPOCHS = 2

#Get Data Function

- returns two arrays X, y 
- `X.shape = (num_images, image_size, image_size, 3)`
- `y.shape = (num_images, image_size, image_size, 1)`

In [10]:
def get_data(num_images=15):
    test_path =[filename for filename in glob.glob(FOLDER_PATH[0])]
    train_path = [filename for filename in glob.glob(FOLDER_PATH[1])]
    
    # limit to 15 images for test
    test_path.sort()
    train_path.sort()
    
    test_path = test_path[:num_images]
    train_path = train_path[:num_images] 

    X = [] # Image
    y = [] # Mask
    
    images_X = []
    images_y = []
    
    for filename in train_path:
      im = Image.open(filename)
      # print(filename)
      images_X.append(im)
      im_resized = im.resize(IMAGE_SIZE)
      X.append(image.img_to_array(im_resized))

    for filename in test_path:
      im = Image.open(filename)
      # print(filename)
      images_y.append(im)
      im_resized = im.resize(IMAGE_SIZE)
      y.append(image.img_to_array(im_resized))
    
    
    X = np.array(X) / 255.
    y = np.array(y) / 255.
    
    return X, y

# Model Class

In [34]:
class ModelClass():

  def __init__(self, X_train, X_test, y_train, y_test, loss='binary_crossentropy'):
    self.loss=loss
    # Split X, y into train and test 
    self.X_train = X_train
    self.X_test = X_test
    self.y_train = y_train
    self.y_test = y_test
    

    self.experiment_name = EXPERIMENT_NAME
      
  def convolution_operation(self, entered_input, filters=64):
            
      # Taking first input and implementing the first conv block
      conv1 = Conv2D(filters, kernel_size = (3,3), padding = "same")(entered_input)
      batch_norm1 = BatchNormalization()(conv1)
      act1 = ReLU()(batch_norm1)

      # Taking first input and implementing the second conv block
      conv2 = Conv2D(filters, kernel_size = (3,3), padding = "same")(act1)
      batch_norm2 = BatchNormalization()(conv2)
      act2 = ReLU()(batch_norm2)

      ### 
      # padding = 'same' to maintain the same shapes

      return act2

  def encoder(self, entered_input, filters=64):
      # Collect the start and end of each sub-block for normal pass and skip connections
      enc1 = self.convolution_operation(entered_input, filters)
      MaxPool1 = MaxPooling2D(strides = (2,2))(enc1)
      return enc1, MaxPool1

  def decoder(self, entered_input, skip, filters=64):
      # Upsampling and concatenating the essential features
      Upsample = Conv2DTranspose(filters, (2, 2), strides=2, padding="same")(entered_input)
      Connect_Skip = Concatenate()([Upsample, skip])
      out = self.convolution_operation(Connect_Skip, filters)
      return out


  def U_Net(self, Image_Size):
      # Take the image size and shape
      input1 = Input(Image_Size)

      # Construct the encoder blocks
      skip1, encoder_1 = self.encoder(input1, 64)
      skip2, encoder_2 = self.encoder(encoder_1, 64*2)
      skip3, encoder_3 = self.encoder(encoder_2, 64*4)
      skip4, encoder_4 = self.encoder(encoder_3, 64*8)

      # Preparing the next block
      conv_block = self.convolution_operation(encoder_4, 64*16)

      # Construct the decoder blocks
      decoder_1 = self.decoder(conv_block, skip4, 64*8)
      decoder_2 = self.decoder(decoder_1, skip3, 64*4)
      decoder_3 = self.decoder(decoder_2, skip2, 64*2)
      decoder_4 = self.decoder(decoder_3, skip1, 64)

      out = Conv2D(1, 1, padding="same", activation="sigmoid")(decoder_4)

      model = Model(input1, out)
      return model

  def set_model(self):
      # Instantiate Model
      input_shape = self.X_train.shape[1:]
      self.model = self.U_Net(input_shape)
      
      # Compile Model
      self.model.compile(loss=self.loss, 
                    optimizer='adam')

  def run(self):

      print(80*'-')
      print('------MODEL RUNNING------')

      # set model
      self.set_model()

      mc = ModelCheckpoint('oxford_segmentation.h5', save_best_only=True)
      
      self.model.fit(self.X_train, self.y_train, validation_split=0.3,
                      batch_size=BATCH_SIZE, epochs=EPOCHS, callbacks=[mc])
      
      self.mlflow_log_param('loss', self.loss)

      print(80*'=')
      print('------MODEL SUCCESFULLY------')
      

  def evaluate(self):
      print(80*'-')
      print('------MODEL EVALUATING------')        
      results = self.model.evaluate(self.X_test, self.y_test)
      self.mlflow_log_metric('loss', results)
      print(80*'=')
      print('------MODEL EVALUATED------')
      
      
  @memoized_property
  def mlflow_client(self):
      mlflow.set_tracking_uri(MLFLOW_URI)
      return MlflowClient()

  @memoized_property
  def mlflow_experiment_id(self):
      try:
          return self.mlflow_client.create_experiment(self.experiment_name)
      except BaseException:
          return self.mlflow_client.get_experiment_by_name(self.experiment_name).experiment_id

  @memoized_property
  def mlflow_run(self):
      return self.mlflow_client.create_run(self.mlflow_experiment_id)

  def mlflow_log_param(self, key, value):
      self.mlflow_client.log_param(self.mlflow_run.info.run_id, key, value)

  def mlflow_log_metric(self, key, value):
      self.mlflow_client.log_metric(self.mlflow_run.info.run_id, key, value)
          

# Testing code

In [12]:
# Get data
X, y = get_data()



In [13]:
X.shape

(15, 160, 160, 3)

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [35]:
# instantiate model 
model = ModelClass(X_train, X_test, y_train, y_test)


In [36]:
model.run()

--------------------------------------------------------------------------------
------MODEL RUNNING------
Epoch 1/2
1/1 [==============================] - 4s 4s/step - loss: 0.9066 - val_loss: 0.7357
Epoch 2/2
1/1 [==============================] - 0s 246ms/step - loss: 0.7988 - val_loss: 0.7371
------MODEL SUCCESFULLY------


In [37]:
model.evaluate()

--------------------------------------------------------------------------------
------MODEL EVALUATING------
1/1 [==============================] - 0s 96ms/step - loss: 0.7293
------MODEL EVALUATED------
